# Dynamic mode decomposition with control on a linear system
We apply dynamic mode decomposition with control (DMD) to a low-dimensional, linear system
(this is example in Sec. 4 in Proctor et al., _"Dynamic Mode Decomposition with Control"_, SIAM 2016):

$$x_{k+1} =\begin{bmatrix} 1.5 & 0\\ 0 & 0.1 \end{bmatrix} + \begin{bmatrix} 1\\ 0 \end{bmatrix} u$$

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

import pykoopman as pk

Integrate system

In [242]:
A = np.matrix([[1.5, 0],[0, 0.1]])
B = np.matrix([[1],[0]])

def advance_linear_system(x0,u,n,A=A,B=B):
    # A = np.matrix([[1.5, 0],[0, 0.1]])
    # B = np.matrix([1,0])
    x = np.zeros([n,len(x0)])
    x[0,:] = x0
    for i in range(n-1):
        x[i+1,:] = A.dot(x[i,:]) + B.dot(u[np.newaxis,i])
    return x

x0 = np.array([4,7])
u = np.array([-4, -2, -1, -0.5, 0, 0.5, 1, 3, 5])
n = len(u)+1
x = advance_linear_system(x0,u,n)
X1 = x[:-1,:]
X2 = x[1:,:]
print('X1 = ', X1)
print('X2 = ', X2)
print('u = ', u)

X1 =  [[4.000000e+00 7.000000e+00]
 [2.000000e+00 7.000000e-01]
 [1.000000e+00 7.000000e-02]
 [5.000000e-01 7.000000e-03]
 [2.500000e-01 7.000000e-04]
 [3.750000e-01 7.000000e-05]
 [1.062500e+00 7.000000e-06]
 [2.593750e+00 7.000000e-07]
 [6.890625e+00 7.000000e-08]]
X2 =  [[2.00000000e+00 7.00000000e-01]
 [1.00000000e+00 7.00000000e-02]
 [5.00000000e-01 7.00000000e-03]
 [2.50000000e-01 7.00000000e-04]
 [3.75000000e-01 7.00000000e-05]
 [1.06250000e+00 7.00000000e-06]
 [2.59375000e+00 7.00000000e-07]
 [6.89062500e+00 7.00000000e-08]
 [1.53359375e+01 7.00000000e-09]]
u =  [-4.  -2.  -1.  -0.5  0.   0.5  1.   3.   5. ]


Apply DMD on the data from the controlled system

In [243]:
U, s, Vh = np.linalg.svd(X1.T, full_matrices=False)
print('U = ', U)
print('s = ', s)
print('V = ', Vh.T)

A = np.dot(X2.T,np.dot(Vh.T*(s**(-1)),U.T))
print('A = ', A)

U =  [[-0.84266992 -0.5384305 ]
 [-0.5384305   0.84266992]]
s =  [9.77868806 5.53737234]
V =  [[-0.73012792  0.67630768]
 [-0.2108914  -0.08794642]
 [-0.09002844 -0.08658323]
 [-0.0434725  -0.04755262]
 [-0.02158208 -0.02420241]
 [-0.03231915 -0.03645275]
 [-0.0915604  -0.10331191]
 [-0.22351418 -0.2522051 ]
 [-0.59379361 -0.67001502]]
A =  [[ 2.17160989e+00 -9.95420579e-01]
 [-1.58023594e-17  1.00000000e-01]]


This is obviously not correct.
So let's apply DMDc on the data from the controlled system with known B

In [246]:
U, s, Vh = np.linalg.svd(X1.T, full_matrices=False)
A = np.dot(X2.T-np.dot(B,u[np.newaxis,:]),np.dot(Vh.T*(s**(-1)),U.T))
print('A = ', A)

A =  [[ 1.50000000e+00 -1.36609474e-17]
 [-1.58023594e-17  1.00000000e-01]]


This yields the correct system matrix. Let's further assume B is also unknown.

In [263]:
r = 3
Omega = np.vstack([X1.T,u[np.newaxis,:]])
U, s, Vh = np.linalg.svd(Omega, full_matrices=False)
Ur = U[:,0:r]
sr = s[0:r]
Vr = Vh[0:r,:].T
# G = np.dot(X2.T,np.dot(Vh.T*(s**(-1)),U.T))
G = np.dot(X2.T,np.dot(Vr*(sr**(-1)),Ur.T))
print('G = ', G)

Aest = G[:,0:2]
Best = G[:,2]

np.allclose(A,Aest)
np.allclose(B,Best[:,np.newaxis])

G =  [[ 1.50000000e+00  4.68917440e-17  1.00000000e+00]
 [-1.32593420e-17  1.00000000e-01  6.88569357e-18]]


True